<a href="https://colab.research.google.com/github/ruby0322/bda-midterm-project/blob/preprocessing/bda_midterm_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 環境設定及整合


### Python 套件導入

In [ ]:
import os
import pandas as pd
import chardet

### 資料集下載及讀取

In [ ]:
!gdown 1jJGJjXcQqRaAWBoHB4hf0PT-OH4vRvvM

Downloading...
From (original): https://drive.google.com/uc?id=1jJGJjXcQqRaAWBoHB4hf0PT-OH4vRvvM
From (redirected): https://drive.google.com/uc?id=1jJGJjXcQqRaAWBoHB4hf0PT-OH4vRvvM&confirm=t&uuid=0c8368d2-041f-481f-b894-7b423a5ea778
To: /content/BDA.zip
100% 470M/470M [00:08<00:00, 55.6MB/s]


In [ ]:
!unzip -P bda2024 ./BDA.zip
!mkdir -p data
!mv ./*.csv ./data

mkdir: cannot create directory ‘./data’: File exists
mv: cannot stat './*.csv': No such file or directory


In [ ]:
root = '/content/data'
data = {}
for filename in os.listdir(root):
    words = filename.split('_')
    readable_filename = ''.join(words[-2:])
    filepath = f'{root}/{filename}'
    print(f'正在載入 {readable_filename} ...')
    df = pd.read_csv(filepath, encoding='utf-8')
    print(df)
    data[readable_filename[:-4]] = df
    print(f'成功載入 {readable_filename}。')

正在載入 微股力籌碼數據-2年.csv ...


<ipython-input-15-f57885bd5221>:13: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath, encoding='utf-8')


         stock_name stock_symbol  foreign_investor_bought  \
0                信大         1109                     2000   
1                信大         1109                        0   
2                信大         1109                    11000   
3                信大         1109                    13000   
4                信大         1109                    68000   
...             ...          ...                      ...   
998026          現觀科         6906                    28000   
998027        金萬林-創         6645                    14000   
998028  群益ESG投等債20+       00937B                  1511400   
998029         玖鼎電力         4588                   389000   
998030      世界健身-KY         2762                    10000   

        foreign_investor_sold  investment_trust_bought  investment_trust_sold  \
0                        3000                        0                      0   
1                       11000                        0                      0   
2                       

<ipython-input-15-f57885bd5221>:13: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath, encoding='utf-8')


        stock_name stock_symbol   open   high    low  close  volume  \
0               日馳         1526  47.55  48.45  47.55  48.30     138   
1               日馳         1526  48.30  48.30  47.40  47.95     153   
2               日馳         1526  48.45  48.70  47.80  48.10     120   
3               日馳         1526  47.95  47.95  47.55  47.60     165   
4               日馳         1526  47.65  47.65  45.30  45.65     514   
...            ...          ...    ...    ...    ...    ...     ...   
1154220         統一         1216  76.20  78.60  76.00  77.20   20607   
1154221         統一         1216  76.80  78.80  76.40  78.80   16466   
1154222         統一         1216  78.00  78.70  77.10  78.20   16015   
1154223         統一         1216  78.00  78.00  76.30  77.40    8524   
1154224         統一         1216  76.10  77.20  75.80  76.20    8347   

                        date  
0        2022-03-01 00:00:00  
1        2022-03-02 00:00:00  
2        2022-03-03 00:00:00  
3        2022-03-04 00:

In [ ]:
data.keys()

dict_keys(['微股力籌碼數據-2年', '微股力財報數據-2年', '內容數據新聞1', '討論數據dcard', '討論數據mobile01-2', '內容數據新聞2', '內容數據新聞3', '微股力個股交易數據-2年', '討論數據ptt', '討論數據mobile01-1', '微股力社群PKTD-2年'])

### 列出各 table 的各行標題名稱

In [12]:
print('All tables:', data.keys(),'\n')
keys_list = list(data.keys())
for i in range(len(data)):
    print('Categories of', keys_list[i], ':', data[keys_list[i]].keys(),'\n')

All tables: dict_keys(['微股力籌碼數據-2年', '微股力財報數據-2年', '內容數據新聞1', '討論數據dcard', '討論數據mobile01-2', '內容數據新聞2', '內容數據新聞3', '微股力個股交易數據-2年', '討論數據ptt', '討論數據mobile01-1', '微股力社群PKTD-2年']) 

Categories of 微股力籌碼數據-2年 : Index(['stock_name', 'stock_symbol', 'foreign_investor_bought',
       'foreign_investor_sold', 'investment_trust_bought',
       'investment_trust_sold', 'dealer_bought', 'dealer_sold', 'date'],
      dtype='object') 

Categories of 微股力財報數據-2年 : Index(['stock_name', 'stock_symbol', 'period', 'gross_profit_margin',
       'operating_profit_margin', 'net_profit_margin', 'return_on_equity',
       'debt_ratio', 'interest_coverage', 'current_ratio', 'quick_ratio',
       'accounts_turnover', 'inventory_turnover', 'eps',
       'book_value_per_share', 'date'],
      dtype='object') 

Categories of 內容數據新聞1 : Index(['id', 'p_type', 's_name', 's_area_name', 'post_time', 'title', 'author',
       'content', 'page_url'],
      dtype='object') 

Categories of 討論數據dcard : Index(['id', 'forum', 

### 依照查詢到的 AI 類股代號，將與股票漲跌有關的 table 進行篩選 （以 yahoo 股市為準）

In [15]:
# AI 類股代號
AI_stock_label = ['1504','2059','2308','2312','2317','2330','2353','2356','2357','2368',
          '2376','2377','2382','2383','2395','2439','2449','2453','2454','3029',
          '3037','3044','3078','3169','3227','3231','3324','3443','3653','3661',
          '4966','5274','5289','5351','5439','6166','6214','6274','6414','6515',
          '6643','6669','6689','6811','6841','6902','6933','8210']

In [16]:
print('AI 類別股個股交易數據')
ai_stock_exchange_data = data['微股力個股交易數據-2年'][data['微股力個股交易數據-2年']['stock_symbol'].isin(AI_stock_label)]
display(ai_stock_exchange_data)

print('AI 類別股社群PKTD')
ai_stock_pktd = data['微股力社群PKTD-2年'][data['微股力社群PKTD-2年']['stock_symbol'].isin(AI_stock_label)]
display(ai_stock_pktd)

print('AI 類別股籌碼數據')
ai_stock_tokens = data['微股力籌碼數據-2年'][data['微股力籌碼數據-2年']['stock_symbol'].isin(AI_stock_label)]
display(ai_stock_tokens)

print('AI 類別股財報數據')
data['微股力財報數據-2年']['stock_symbol'] = data['微股力財報數據-2年']['stock_symbol'].astype(str, errors='ignore')
ai_stock_fin = data['微股力財報數據-2年'][data['微股力財報數據-2年']['stock_symbol'].isin(AI_stock_label)]
display(ai_stock_fin)


AI 類別股個股交易數據


,stock_name,stock_symbol,open,high,low,close,volume,date
645379,東元,1504,54.7,55.6,52.7,53.5,23449,2023-06-27 00:00:00
645380,東元,1504,53.8,54.1,52.5,52.7,17530,2023-06-28 00:00:00
645381,東元,1504,52.9,53.5,52.5,52.9,10579,2023-06-29 00:00:00
645382,東元,1504,53.1,53.7,51.9,53.3,12575,2023-06-30 00:00:00
645383,東元,1504,53.9,54.8,53.5,54.0,14057,2023-07-03 00:00:00
...,...,...,...,...,...,...,...,...
1127996,穎崴,6515,806.0,812.0,781.0,785.0,1604,2023-06-16 00:00:00
1127997,穎崴,6515,785.0,808.0,770.0,796.0,1192,2023-06-19 00:00:00
1127998,穎崴,6515,799.0,805.0,772.0,775.0,1065,2023-06-20 00:00:00
1127999,穎崴,6515,780.0,784.0,767.0,770.0,735,2023-06-21 00:00:00


AI 類別股社群PKTD


,date,stock_symbol,actionP,actionK,actionT,actionD
36,2022-01-01 00:53,2449,0,123,0,0
41,2022-01-01 00:54,2449,0,0,123,0
63,2022-01-01 01:22,2357,123,0,0,0
82,2022-01-01 01:39,2308,0,0,123,0
102,2022-01-01 02:01,2368,0,0,123,0
...,...,...,...,...,...,...
10436687,2023-12-31 14:05,2330,0,123,0,0
10436688,2023-12-31 14:05,2330,123,0,0,0
10436701,2023-12-31 14:26,2454,0,123,0,0
10436702,2023-12-31 14:27,2454,123,0,0,0


AI 類別股籌碼數據


,stock_name,stock_symbol,foreign_investor_bought,foreign_investor_sold,investment_trust_bought,investment_trust_sold,dealer_bought,dealer_sold,date
628392,東元,1504,4859400,4889000,0,1036619,299000,779228,2023-07-11 00:00:00
628393,東元,1504,4144000,8702050,5000,58000,955786,309100,2023-07-10 00:00:00
628394,東元,1504,6856032,4315000,65000,1481808,159394,562960,2023-07-07 00:00:00
628395,東元,1504,6132048,4460588,167000,1179000,483000,408153,2023-07-06 00:00:00
628396,東元,1504,3975000,4124000,7000,2751174,122000,163428,2023-07-05 00:00:00
...,...,...,...,...,...,...,...,...,...
997656,雙鴻,3324,928000,955000,136000,125000,334095,235541,2024-02-26 00:00:00
997887,AMAX-KY,6933,12000,32000,0,0,2000,0,2024-02-26 00:00:00
997908,走著瞧-創,6902,0,0,0,0,0,0,2024-02-26 00:00:00
997937,伊雲谷,6689,184015,72000,2000,0,3000,180,2024-02-26 00:00:00


AI 類別股財報數據


,stock_name,stock_symbol,period,gross_profit_margin,operating_profit_margin,net_profit_margin,return_on_equity,debt_ratio,interest_coverage,current_ratio,quick_ratio,accounts_turnover,inventory_turnover,eps,book_value_per_share,date
461,東元,1504,Q,24.35,11.39,11.70,1.96,32.51,37.09,201.10,135.21,1.30,0.88,0.78,40.37,2023-03-01 00:00:00
462,東元,1504,Q,22.98,8.78,7.47,1.34,31.82,34.54,222.11,149.00,1.30,0.88,0.48,37.94,2022-12-01 00:00:00
464,東元,1504,Q,22.54,8.87,8.38,1.53,33.72,28.25,224.43,146.93,1.26,0.87,0.51,35.61,2022-09-01 00:00:00
466,東元,1504,Q,22.97,8.85,8.05,1.44,34.95,32.33,199.52,131.74,1.28,0.86,0.48,35.45,2022-06-01 00:00:00
467,東元,1504,Q,21.90,8.28,3.27,0.51,34.49,16.32,198.24,131.08,1.25,0.86,0.17,35.91,2022-03-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16367,川湖,2059,Y,61.28,50.12,46.93,16.86,20.14,113.88,745.17,696.78,3.34,2.31,28.38,172.47,2023-12-01 00:00:00
16384,宜鼎,5289,Q,33.21,16.62,12.30,3.74,23.70,133.88,377.70,302.03,1.59,1.25,2.97,79.61,2023-12-01 00:00:00
16385,宜鼎,5289,Y,34.23,16.60,13.76,16.04,23.70,202.65,377.70,302.03,6.03,4.72,12.98,79.61,2023-12-01 00:00:00
16472,欣興,3037,Q,17.55,5.41,11.36,3.05,55.42,25.30,195.67,163.73,1.23,1.99,1.91,59.49,2023-12-01 00:00:00


In [17]:
# 檢查數量是否正確
print(ai_stock_exchange_data['stock_symbol'].nunique())
print(ai_stock_pktd['stock_symbol'].nunique())
print(ai_stock_tokens['stock_symbol'].nunique())
print(ai_stock_fin['stock_symbol'].nunique())

48
48
48
48
